# 2 Advanced Forecasting Pipelines

In the previous notebook, we considered
* sequential pipelines
* tuning their hyperparameters

This notebook is about:
* tuning the structure of sequential pipelines
* introducing graphical pipelines





In [ ]:
import warnings
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head, load_longley, load_macroeconomic
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    ColumnEnsembleForecaster,
    ForecastX,
    MultiplexForecaster,
    make_reduction,
)
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.sarimax import SARIMAX
from sktime.performance_metrics.forecasting import mean_absolute_error
from sktime.pipeline.pipeline import Pipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.exponent import ExponentTransformer
from sktime.transformations.series.subset import ColumnSelect
warnings.filterwarnings("ignore")

## 2.1 Tuning the Pipeline's Structure (AutoML)
In the previous notebook, we performed a grid search to find the best hyperparameters. However, also the structure of the pipeline is a hyperparamter. Thus, for tuning this hyperparameter, `sktime` contains
* MultiplexForecasters
* OptionalPassthrough
* ...

That are explained in the following before showing an AutoML example.


### 2.1.1 Tuning ComponentsModel selection

Pipeline structure choices influence performance

`sktime` allows to expose these choices via structural compositors:

* switch between transform/forecast: `MultiplexTransformer`, `MultiplexForecaster`
* transformer on/off: `OptionalPassthrough`
* sequence of transformers: `Permute`

Combine with pipelines and `FeatureUnion` for rich structure space

#### `MultiplexForecaster`
We can use the `MultiplexForecaster` to compare the performance of different forecasters. This approach might be useful if we want to compare the performance of different forecasters that have been tuned and fitted already separately. The `MultiplexForecaster` is just a forecaster compostition that provides a parameters `selected_forecaster: List[str]` that can be tuned with a grid search. The other parameters of the forecasters are not tuned.


In [ ]:
from sktime.forecasting.compose import MultiplexForecaster

forecaster = MultiplexForecaster(
    forecasters=[
        ("naive", NaiveForecaster()),
        ("stl", STLForecaster()),
        ("theta", ThetaForecaster()),
    ]
)
gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid={"selected_forecaster": ["naive", "stl", "theta"]},
    cv=cv,
    n_jobs=-1,
)
gscv.fit(y)
gscv.best_params_

{'selected_forecaster': 'theta'}

#### Optional Passthrough

In `sktime` there is a transformer composition called `OptionalPassthrough()` which gets a transformer as an argument and a param `passthrough: bool`. Setting `passthrough=True` will return an identity transformation for the given data. Setting `passthrough=False` will apply the given inner transformer on the data.

In [ ]:
from sktime.transformations.series.compose import OptionalPassthrough

transformer = OptionalPassthrough(transformer=Detrender(), passthrough=True)
transformer.fit_transform(y_train).head()

Period
1991-01    266.0
1991-02    145.9
1991-03    183.1
1991-04    119.3
1991-05    180.3
Freq: M, Name: Number of shampoo sales, dtype: float64

In [ ]:
y_train.head()

Period
1991-01    266.0
1991-02    145.9
1991-03    183.1
1991-04    119.3
1991-05    180.3
Freq: M, Name: Number of shampoo sales, dtype: float64

In [ ]:
transformer = OptionalPassthrough(transformer=Detrender(), passthrough=False)
transformer.fit_transform(y_train).head()

Period
1991-01    130.376344
1991-02      1.503263
1991-03     29.930182
1991-04    -42.642900
1991-05      9.584019
Freq: M, Name: Number of shampoo sales, dtype: float64

#### Permutation of transformers

Given a set of four different transformers, we would like to know which permutation (ordering) of the four transformers is having the best error. In total there are `4! = 24` different permutations. We can use the `GridSearchCV` to find the best permutation.

In [ ]:
from sktime.forecasting.compose import Permute

SyntaxError: invalid syntax (1459108173.py, line 2)

In [ ]:
forecaster = TransformedTargetForecaster(
    steps=[
        ("detrender", Detrender()),
        ("deseasonalizer", Deseasonalizer()),
        ("power", TabularToSeriesAdaptor(PowerTransformer())),
        ("scaler", TabularToSeriesAdaptor(RobustScaler())),
        ("forecaster", ExponentialSmoothing()),
    ]
)

param_grid = {
    "permutation": [
        ["detrender", "deseasonalizer", "power", "scaler", "forecaster"],
        ["power", "scaler", "detrender", "deseasonalizer", "forecaster"],
        ["scaler", "deseasonalizer", "power", "detrender", "forecaster"],
        ["deseasonalizer", "power", "scaler", "detrender", "forecaster"],
    ]
}
permuted = Permute(estimator=forecaster, permutation=None)

gscv = ForecastingGridSearchCV(
    forecaster=permuted,
    param_grid=param_grid,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    scoring=MeanSquaredError(square_root=True),
)

permuted = gscv.fit(y, fh=fh)

In [ ]:
permuted.cv_results_

In [ ]:
gscv.best_params_

In [ ]:
# worst params
gscv.cv_results_.sort_values(by="mean_test_MeanSquaredError", ascending=True).iloc[-1][
    "params"
]

## 2.1.2 AutoML and Forecasting

Taking all incredients from above examples, we can build a forecaster that comes close to what is usually called [AutoML](https://en.wikipedia.org/wiki/Automated_machine_learning).
With AutoML we aim to automate as many steps of an ML model creation as possible. The main compositions from `sktime` that we can use for this are:
- `TransformedTargetForecaster`
- `ForecastingPipeline`
- `ForecastingGridSearchCV`
- `OptionalPassthrough`
- `Permute` [Not Covered in this workshop]

### Univariate example
Please see appendix section for an example with exogenous data

In [ ]:
pipe_y = TransformedTargetForecaster(
    steps=[
        ("detrender", OptionalPassthrough(Detrender())),
        ("deseasonalizer", OptionalPassthrough(Deseasonalizer())),
        ("scaler", OptionalPassthrough(TabularToSeriesAdaptor(RobustScaler()))),
        ("forecaster", STLForecaster()),
    ]
)
permuted_y = Permute(estimator=pipe_y, permutation=None)

param_grid = {
    "permutation": [
        ["detrender", "deseasonalizer", "scaler", "forecaster"],
        ["scaler", "deseasonalizer", "detrender", "forecaster"],
    ],
    "estimator__detrender__passthrough": [True, False],
    "estimator__deseasonalizer__passthrough": [True, False],
    "estimator__scaler__passthrough": [True, False],
    "estimator__scaler__transformer__transformer__with_scaling": [True, False],
    "estimator__scaler__transformer__transformer__with_centering": [True, False],
    "estimator__forecaster__sp": [4, 8, 12],
}

gscv = ForecastingGridSearchCV(
    forecaster=permuted_y,
    param_grid=param_grid,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    scoring=MeanSquaredError(square_root=True),
    error_score="raise",
)

gscv.fit(y=y_train, fh=fh)

In [ ]:
gscv.cv_results_["mean_test_MeanSquaredError"].min()

In [ ]:
gscv.cv_results_["mean_test_MeanSquaredError"].max()

## 3.2 Graphical Pipeline

### What are Graphical Pipelines?
Recap sequential pipelines:

<img src="img/sequential_pipeline.png" width=750 />

Many tasks are non-sequential. To solve this two possibilities exist:
1. Nesting Sequential Pipelines.
2. Using Graphical Pipelines.


Thus, there is the generalised graphial pipeline.
* Graphical means that different steps may share the same predecessor or provide their outputs to the same successor (the dataflows can branch and merge).
<img src="img/graphical_pipeline.png" width=750 />


* Generalised means that the pipeline can be used for multiple tasks (e.g. forecasting, classification, ...).

**Note**

The graphical pipeline is still experimental. Thus, this graphical should not used in production. However, we would be happy to get feedback on the graphical pipeline.



### Potential Use-Cases
There exist various potential use-case for the graphical pipeline. In the following, we focus on a forecasting and a classification pipeline.
#### Forecasting Use-Case for Graphical Pipelines


The input of forecasters depends on the output of other forecasters, which same the same input.
* Forecaster could use the same preprocessing (branching of data flow)
* Forecaster could use outputs of multiple predeccessors (merging of data flow)

<img src="img/graphical_pipeline_example.png" width=900 />


#### Classification Use-Case for Graphical Pipelines


The input of classifier are different features and not all of are observable. Idea: Use a soft sensor: I.e., a regressor for the non-observable features.

<img src="img/graphical_pipeline_softsensor.png" width=900 />

**Note:** The current experimental state of the graphical pipeline does not fully support this use-case. However, we are working on this. If you are interested in this use-case and want to contribute, please contact us.

### Credits
The graphical pipeline was first developed by pyWATTS [1] and was then adapted for sktime. The original implementation can be found [pyWATTS](https://github.com/KIT-IAI/pyWATTS). pyWATTS is a open source library developed at the Institute of Applied Informatics and Automation at the KIT and funded by HelmholtzAI.

> [1] Heidrich, Benedikt, et al. "pyWATTS: Python workflow automation tool for time series." arXiv preprint arXiv:2106.10157 (2021).



### 3.2.1 Constructing simple Graphical Pipelines for Forecasting

Two Ways to create graphical pipelines:

1. Pass all steps to the pipeline during initialisation as for the sequential pipeline.

## 3.3 Pipeline Diagnostics

In [ ]:
differencer = Differencer()

general_pipeline = Pipeline(
    [
        {"skobject": differencer, "name": "differencer", "edges": {"X": "y"}},
        {
            "skobject": SARIMAX(),
            "name": "sarimax",
            "edges": {"X": "X", "y": "differencer"},
        },
        {
            "skobject": differencer,
            "name": "differencer_inv",
            "edges": {"X": "sarimax"},
            "method": "inverse_transform",
        },
    ]
)

The pipeline can be visualised as follows:
<img src="img/forecasting_pipeline.png" width=750 />


In [ ]:
general_pipeline = Pipeline()
differencer = Differencer()

general_pipeline = general_pipeline.add_step(
    differencer, "differencer", edges={"X": "y"}
)
general_pipeline = general_pipeline.add_step(
    SARIMAX(), "sarimax", edges={"X": "X", "y": "differencer"}
)
general_pipeline = general_pipeline.add_step(
    differencer, "differencer_inv", edges={"X": "sarimax"}, method="inverse_transform"
)

#### Summary of the arguments:
The `add_step`'s parameter or key of the dicts in the step list during initialisation are:

* skobject: The sktime object added to the pipeline
* name: The name of the step
* edges: The keys of the dictionary indicate the input of the skobject (X or y), and the values are the names of the steps that should be connected to the input argument. Note subsetting using `__` and feature union via lists are supported.
* method: The skobject's method that should be called. If not provided, the default method would be inferred based on the added skobject. This parameter is used for the inverse_transform method. Optional.
* kwargs: Additional keyword arguments passed to the sktime object. Optional.

Fitting and predicting works as for any sktime object

In [ ]:
y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

general_pipeline.fit(y=y_train, X=X_train, fh=[1, 2, 3, 4])
general_pipeline.predict(X=X_test)

### 3.2.2 Constructing Graphical Pipelines for Classification

In [ ]:
# TODO: Graphical Pipeline by initialisation

Again, also using `add_step` is possible:

general_pipeline = Pipeline()
general_pipeline = general_pipeline.add_step(
    ExponentTransformer(), "exponent", edges={"X": "X"}
)
general_pipeline = general_pipeline.add_step(
    KNeighborsTimeSeriesClassifier(), "classifier", edges={"X": "exponent", "y": "y"}
)

This pipeline can be visualised as follows:

<img src="img/classification_pipeline.png" width=750 />


In [ ]:
X, y = load_arrow_head(split="train", return_X_y=True)
general_pipeline.fit(X=X, y=y)
general_pipeline.predict(X=X)

### 3.2.3 Becoming more complex
The considered use-case is to forecast the inflation using forecasts of the real gross domestic product, real disposable personal income, and the unemployment rate. Furthermore the unemployment rate is forecasted using the same features except the unemployment rate itself.

<img src="img/graphical_pipeline_example.png" width=750 />


The data is taken from the macrodata dataset from the statsmodels package.

**Note** We stick with the `add_step` in the following.


In [ ]:
pipeline = Pipeline()
sklearn_scaler = StandardScaler()
sktime_scaler = TabularToSeriesAdaptor(sklearn_scaler)
deseasonalizer = Deseasonalizer(sp=4)

pipeline = pipeline.add_step(
    sktime_scaler, name="scaler", edges={"X": "X__realgdp_realdpi_unemp"}
)
pipeline = pipeline.add_step(
    deseasonalizer, name="deseasonalizer", edges={"X": "X__realgdp_realdpi"}
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_gdp",
    edges={"y": "deseasonalizer__realgdp"},
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_dpi",
    edges={"y": "deseasonalizer__realdpi"},
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_unemp",
    edges={
        "y": "scaler__unemp",
        "X": [
            "forecaster_gdp",
            "forecaster_dpi",
        ],
    },
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_inflation",
    edges={"X": ["forecaster_dpi", "forecaster_unemp"], "y": "y"},
)

In [ ]:
data = load_macroeconomic()

X = data[["realgdp", "realdpi", "unemp"]]
y = data[["infl"]]
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X=X, fh=fh)
X_train

In [ ]:
pipeline.fit(y=y_train, X=X_train, fh=fh)
result = pipeline.predict(X=None, fh=y_test.index)
result

#### But what are the best hyperparameters?

* which forecaster should be used for which variable -> `MultiplexForecaster`
* what should be the hyperparameters of the forecaster
* which features should be used for the different forecasters -> Tune the edges of the graphical pipeline!

<img src="img/graphical_pipeline_example_grid.png" width=900 />

**Idea:** Combine graphical pipeline with ForecastingGridSearchCV.

1. Specify the parameter grid:

The keys of the dictionary are the parameters' in the pipeline, and the values specify which options should be tested.
Keys have the following structure: parameter of a step `<step_name>__skobject__<parameter-name>` and input edges of a step `<step-name>__edges_<Xory>`.

In [ ]:
param_grid = {
    "forecaster_inflation__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_unemp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_dpi__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_gdp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_inflation__edges__X": [
        ["forecaster_unemp"],
        ["forecaster_unemp", "forecaster_dpi"],
    ],
    "forecaster_unemp__edges__X": [
        [],
        ["forecaster_dpi"],
        ["forecaster_gdp", "forecaster_dpi"],
    ],
    "deseasonalizer__edges__X": ["X__realgdp_realdpi", "scaler__realgdp_realdpi"],
}

Initialise the gridsearch using pipeline, cross-validation strategy, scoring, and param_grid.


In [ ]:
gridcv = ForecastingGridSearchCV(
    pipeline,
    cv=SlidingWindowSplitter(
        window_length=len(X_train) - 20,
        step_length=4,
        fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    ),
    scoring=mean_absolute_error,
    param_grid=param_grid,
)

Call fit on the gridsearch object.


In [ ]:
gridcv.fit(y=y_train, X=X_train)

In [ ]:
gridcv.cv_results_

In [ ]:
gridcv.best_params_

In [ ]:
result = gridcv.predict(X=None, fh=y_test.index)
result

### How does the graphical pipeline compare to the sequential pipeline?

Let us try to implement a simplified version of the above example using sequential pipelines with nesting.
<img src="img/graphical_pipeline_simplified.png" width=900 />


Create sequential pipelines for forecasting the GDP, DPI and unemployment rate.


In [ ]:
forecasting_pipeline_gdp = (
    ColumnSelect(["realgdp"])  # To train the forecaster only on the realgdp column
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)
forecasting_pipeline_dpi = (
    ColumnSelect(["realdpi"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)

forecasting_pipeline_unemp = (
    ColumnSelect(["unemp"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)

Use ColunmEnsembleForecaster to combine the forecasts of the DPI, GDP, UNEMP. (Union of forecasts)

In [ ]:
input_inflation_forecast = ColumnEnsembleForecaster(
    [
        ("realdpi", forecasting_pipeline_dpi, "realdpi"),
        ("realgdp", forecasting_pipeline_gdp, "realgdp"),
        ("unemp", forecasting_pipeline_unemp, "unemp"),
    ]
)

Create the inflation forecaster.

In [ ]:
inflation_forecast = ForecastX(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    input_inflation_forecast,
)

In [ ]:
inflation_forecast.fit(y=y_train, X=X_train, fh=fh)

In [ ]:
inflation_forecast.predict()

Ok, we can built it, but what about the hyperparameter tuning? Let us investigate the paramters..

In [ ]:
inflation_forecast.get_params(True)

# Uff...


### Advantages of graphical pipelines
* Enable an easy implementation of complex pipelines
    * By nesting sequential pipelines, even a simplified version of the graphical pipeline is very complicat to implement.
    * By nesting sequential pipelines, some graphical pipelines are not possible to implement (e.g., the example with coupled ForecastX).
* Preprocessing steps can not be shared between the different forecasters.
* The parameter structure can be very complex for the sequential pipelines.
* In a complex scenario, how would you fine-tune the edges?

### Advantages of sequential pipelines
* Constructing simple pipelines is very easy.
* Inverse operations are automatically applied.
* This is a mature feature compared to the experimental graphical pipeline.


---

### Credits: notebook 2 - pipelines

notebook creation: benheid (graphical pipeline, overall), AutoML is based on pyData global 2022 notebook created by aiwalter.

forecaster pipelines: fkiraly, aiwalter\
transformer pipelines & compositors: fkiraly, mloning, miraep8\
dunder interface: fkiraly, miraep8\
graphical pipeline: benheid, fkiraly\

tuning, autoML: mloning, fkiraly, aiwalter\
CV and splitters: mloning, kkoralturk, khrapovs\
forecasting metrics: mloning, aiwalter, rnkuhns, fkiraly\
backtesting, evaluation: aiwalter, mloning, fkiraly, topher-lo